# 3D Vision Homework 1
>王子轩 2023011307
`wang-zx23@mails.tsinghua.edu.cn`
## Probelem 1
### Subproblem 1.1
The map $\textit{f}: \mathbb{R}^n \rightarrow \mathbb{R}^3$ is defined by 
$$
f(u, v) = \begin{bmatrix}
a \cos u \sin v  & \\
b \sin u  \sin v  & \\
c \cos v &  
\end{bmatrix}
\text{where} - \pi \leq u \leq  \pi, \quad 0 \leq v \leq \pi
$$
The function $f$ maps the 2D domain to an ellipsoid.
Let $a = 1$, $b = 1$, $c = \frac{1}{2}$. Let $\mathbf{p} = (u, v)$ be a point in the domain of $f$, and let
$\gamma: (-1, 1) \rightarrow \mathbb{R}^2$ be a curve with $\gamma(0) = \mathbf{p}$ and $\gamma'(t) = \mathbf{v}$.
Now we let $\mathbf{p} = (\frac{\pi}{4}, \frac{\pi}{6})$ and $\mathbf{v} = (1, 0)$, we draw the curve $f(\gamma(t))$.

### Solution:
From $\gamma'(t) = (1, 0)\text{ and } \gamma(0) = (\frac{\pi}{4}, \frac{\pi}{6}) $ we get $\gamma(t) = (\frac{\pi}{4} + t, \frac{\pi}{6}) $, thus $$f(\gamma(t)) = \begin{bmatrix}
\cos(\frac{\pi}{4} + t) \sin(\frac{\pi}{6}) \\
\sin(\frac{\pi}{4} + t) \sin(\frac{\pi}{6}) \\
\frac{1}{2} \cos(\frac{\pi}{6})
\end{bmatrix} $$
>Full code using object oriented programming in Python is in src/draw.py 

In [10]:

! python src/draw.py

>Simpler view of the code is list as follows, and the result can be seen in the image.

![draw ellipsoid and curve](assets/image-20250317220956088.png)

In [19]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
a, b, c = 1, 1, 0.5
p = (np.pi/4, np.pi/6)
v = (1, 0)
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Ellipsoid with Curve")
u = np.linspace(-np.pi, np.pi, 50)
v = np.linspace(0, np.pi, 25)
u_grid, v_grid = np.meshgrid(u, v)
x = a * np.cos(u_grid) * np.sin(v_grid)
y = b * np.sin(u_grid) * np.sin(v_grid)
z = c * np.cos(v_grid)
vertices = []
for i in range(len(v)):
    for j in range(len(u)):
        vertices.append([x[i, j], y[i, j], z[i, j]])
vertices = np.array(vertices)

triangles = []
for i in range(len(v)-1):
    for j in range(len(u)-1):
        idx1 = i * len(u) + j
        idx2 = i * len(u) + (j + 1)
        idx3 = (i + 1) * len(u) + j
        idx4 = (i + 1) * len(u) + (j + 1)
        triangles.append([idx1, idx2, idx3])
        triangles.append([idx2, idx4, idx3])
triangles = np.array(triangles)

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(triangles)
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.8, 0.8, 0.8]) 

mesh_wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
mesh_wireframe.paint_uniform_color([0.5, 0.5, 0.5])  

t_values = np.linspace(-1, 1, 100)
curve_points = []

for t in t_values:
    u_t = np.pi/4 + t
    v_t = np.pi/6
    x_t = a * np.cos(u_t) * np.sin(v_t)
    y_t = b * np.sin(u_t) * np.sin(v_t)
    z_t = c * np.cos(v_t)
    
    curve_points.append([x_t, y_t, z_t])

curve_points = np.array(curve_points)

line_indices = [[i, i+1] for i in range(len(curve_points)-1)]
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(curve_points)
line_set.lines = o3d.utility.Vector2iVector(line_indices)
line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in range(len(line_indices))])  # Red color

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])

vis.add_geometry(mesh)
vis.add_geometry(mesh_wireframe)  
vis.add_geometry(line_set)
vis.add_geometry(coordinate_frame)  

opt = vis.get_render_option()
opt.background_color = np.array([1, 1, 1])  
opt.point_size = 5.0
opt.mesh_show_wireframe = True  

vis.run()
vis.destroy_window()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(curve_points)
pcd.paint_uniform_color([0, 1, 0]) 

sample_points = mesh.sample_points_uniformly(number_of_points=3000)
sample_points.paint_uniform_color([0, 0, 0.8])  

o3d.visualization.draw_geometries([mesh, mesh_wireframe, pcd, coordinate_frame, sample_points], 
                                 window_name="Ellipsoid with Curve Points",
                                 mesh_show_wireframe=True)

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


### Subproblem 1.2
Differential map: The differential of a function f at a point p is a linear
map. Equivalent to how we define the differential in class, we can also
define by the gradient of the curve w.r.t. the curve parameter: $Df_{\mathbf{p}(v)} =
f(γ(t))′|t=0.$
### Solution:
#### (a) $Df_{\mathbf{p}(v)}$'s expression:
$Df_{\mathbf{p}(v)} = f(γ(t))′|t=0 = ( f \circ \gamma)'(t)  = f'(\gamma(t)) \gamma'(t) |t=0 = (\frac{\partial f}{\partial \mathbf{p}}) \mathbf{v} = \nabla f \mathbf{v}  
=  (\frac{\partial f}{\partial u}, \frac{\partial f}{\partial v}) \mathbf{v}$ 
thus
$$ 
Df_{\mathbf{p}} 
= 
\begin{bmatrix}
\frac{\partial f}{\partial u} & \frac{\partial f}{\partial v}
\end{bmatrix}
= \begin{bmatrix}
-a\sin u \sin v & a\cos u \cos v \\
b\cos u \sin v & b\sin u \cos v \\
0 & -c\sin v
\end{bmatrix}
= 
\begin{bmatrix}
-\sin u \sin v & \cos u \cos v \\
\cos u \sin v & \sin u \cos v \\
0 & -\frac{1}{2}\sin v
\end{bmatrix}
$$
#### (b) Describe the geometry meaning of the differential map
The differential map $Df_{\mathbf{p}}$ is a linear map that provides the linear approximation of $f$ near point $\mathbf{p}$. It maps 2D tangent space from the input space to the 3D tangent output space, showing how local directions change under the differential map. For example if a point moves at $\mathcal{p}$ by  the velocity of $\mathcal{v}$ in the input space, $Df_{\mathbf{p}}(v)$ represents the velocity of its image point $f_p$ at that in the output space.
#### (c) Draw $\mathcal{Df_p(v)}$ on the ellipsoid
$\mathbf{p}=( \frac{π}{4}, \frac{π}{6}) and \mathbf{v} = (1, 0)$ and we konw that 
$Df_{\mathbf{p}} =
\begin{bmatrix}
-\sin u \sin v & \cos u \cos v \\
\cos u \sin v & \sin u \cos v \\
0 & -\frac{1}{2}\sin v
\end{bmatrix}$ Thus 
$$
Df_{\mathbf{p}} \mathbf{v} 
= \begin{bmatrix}
-\sin u \sin v & \cos u \cos v \\
\cos u \sin v & \sin u \cos v \\
0 & -\frac{1}{2}\sin v
\end{bmatrix}
\begin{bmatrix}
1 \\
0
\end{bmatrix}
= 
\begin{bmatrix}
-\sin u \sin v \\
\cos u \sin v \\
0
\end{bmatrix}
$$
The implemention of full code is in `src/draw.py`, and the class `DifferentialMap` is used to draw the $D_f(v)$ in both directions of the tangent plane.
Run `! python draw.py` you can see the result that the $D_f(v)$ is drawn in both directions of the tangent plane. The Red arrow is the direction of $\mathbf{v} = (1,0)$ and the Blue arrow is the direction of $\mathbf{v} = (0, 1)$.  In the following image, the Red arrow is the direction of $\mathbf{v} = (1,0)$ and the Blue arrow is the direction of $\mathbf{v} = (0, 1)$. See the first image(a):
a
![a](assets\image-20250318001837911.png)

c![c](assets\image-20250318002530291.png)

>Implementation code for the $\mathcal{Df_p}$

```python
class DifferentialMap:
    def __init__(self, ellipsoid, u, v):
        self.ellipsoid = ellipsoid
        self.u = u
        self.v = v
        
    def compute_dfp(self):
        """Calculate Df_p @ (u,v)"""
        a, b, c = self.ellipsoid.a, self.ellipsoid.b, self.ellipsoid.c
        u, v = self.u, self.v
        dfp = np.array([
            [-a * np.sin(u) * np.sin(v), a * np.cos(u) * np.cos(v)],
            [b * np.cos(u) * np.sin(v), b * np.sin(u) * np.cos(v)],
            [0, -c * np.sin(v)]
        ])
        return dfp
    
    def apply_dfp_to_vector(self, v_direction):
        """Apply Df_p to a specific direction vector v"""
        dfp = self.compute_dfp()
        v_direction = np.array(v_direction)
        v_direction = v_direction / np.linalg.norm(v_direction)
        result = dfp @ v_direction
        return result
    
    def create_differential_vector(self, v_direction, scale=0.2, color=[1, 0, 0]):
        """Create a visualization of Df_p(v) for a specific direction v"""
        x = self.ellipsoid.a * np.cos(self.u) * np.sin(self.v)
        y = self.ellipsoid.b * np.sin(self.u) * np.sin(self.v)
        z = self.ellipsoid.c * np.cos(self.v)
        base_point = np.array([x, y, z])
        direction = self.apply_dfp_to_vector(v_direction)
        direction = direction / np.linalg.norm(direction) * scale
        cylinder = o3d.geometry.TriangleMesh.create_cylinder(
            radius=0.01,
            height=np.linalg.norm(direction)
        )
        cone = o3d.geometry.TriangleMesh.create_cone(
            radius=0.02,
            height=0.05
        )
        z_axis = np.array([0, 0, 1])
        if not np.allclose(direction, z_axis) and not np.allclose(direction, -z_axis):
            rotation_axis = np.cross(z_axis, direction)
            rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
            angle = np.arccos(np.dot(z_axis, direction/np.linalg.norm(direction)))
            R = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * angle)
            cylinder.rotate(R, center=[0, 0, 0])
            cone.rotate(R, center=[0, 0, 0])
        cylinder.translate(base_point + direction/2)
        cone.translate(base_point + direction)
        cylinder.paint_uniform_color(color)
        cone.paint_uniform_color(color)
        return [cylinder, cone]
    
    def create_differential_vectors(self, scale=0.2, directions=None):
        """Create visualizations for multiple direction vectors"""
        if directions is None:
            # Default: standard basis vectors
            directions = [
                ([1, 0], [1, 0, 0]),  # u direction, red color
                ([0, 1], [0, 0, 1])   # v direction, blue color
            ]
        arrows = []
        for v_dir, color in directions:
            arrow_parts = self.create_differential_vector(v_dir, scale, color)
            arrows.extend(arrow_parts)
        
        return arrows
```

In [2]:
! python src/draw.py

#### (d) Calculate the normal vector of tagent plane at point $p$.
$$
N(u, v) = \frac{f_u \times f_v}{\| f_u \times f_v \|}\\
 \text{where} f_u = \frac{\partial f}{\partial u} \quad  f_v = \frac{\partial f}{\partial v}
$$
From 
$$
\mathbf{p} = (\frac{\pi}{4},\frac{\pi}{6}), \quad f_u 
= \begin{bmatrix}
-\sin u \sin v\\
\cos u \sin v  \\
0
\end{bmatrix}
f_v = \begin{bmatrix}
\cos u \cos v \\
\sin u \cos v \\
-\frac{1}{2} \sin v
\end{bmatrix}
$$ 
we have 
$f_u \times f_v = \begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
-\sin u \sin v & \cos u \sin v & 0 \\
\cos u \cos v & \sin u \cos v & -\frac{1}{2} \sin v
\end{vmatrix}$

$= \begin{bmatrix}
-\frac{1}{2}\cos u \sin^2 v \
-\frac{1}{2}\sin u \sin^2 v \
-\sin v \cos v
\end{bmatrix}$

$u = \frac{\pi}{4}, v = \frac{\pi}{6}$：

$f_u \times f_v = \begin{bmatrix}
-\frac{1}{2}\frac{\sqrt{2}}{2} \cdot (\frac{1}{2})^2 \
-\frac{1}{2}\frac{\sqrt{2}}{2} \cdot (\frac{1}{2})^2 \
-\frac{1}{2} \cdot \frac{\sqrt{3}}{2}
\end{bmatrix} = \begin{bmatrix}
-\frac{\sqrt{2}}{16} \
-\frac{\sqrt{2}}{16} \
-\frac{\sqrt{3}}{4}
\end{bmatrix}$
$|f_u \times f_v| = \sqrt{(\frac{\sqrt{2}}{16})^2 + (\frac{\sqrt{2}}{16})^2 + (\frac{\sqrt{3}}{4})^2}$
$= \sqrt{\frac{1}{128} + \frac{1}{128} + \frac{3}{16}}$
$= \sqrt{\frac{1}{64} + \frac{12}{64}}$
$= \sqrt{\frac{13}{64}} = \frac{\sqrt{13}}{8}$

However, we need to orient the closed surface outward.
Thus


$N_{(\frac{\pi}{4},\frac{\pi}{6})} = \begin{bmatrix}
\frac{\sqrt{2}}{16} \\
\frac{\sqrt{2}}{16} \\
\frac{\sqrt{3}}{4}
\end{bmatrix} \cdot \frac{8}{\sqrt{13}} = \begin{bmatrix}
\frac{\sqrt{2}}{2\sqrt{13}} \\
\frac{\sqrt{2}}{2\sqrt{13}} \\
\frac{2\sqrt{3}}{\sqrt{13}}
\end{bmatrix}
= [0.19611162,0.19611162, 0.96076892]^T$
How
#### (e) Give a group of orthonormal bases of the tangent space at $f(p)$ and draw it.
Actually, at (c) I have got the orthonormal bases of the tangent space at $f(p)$, that is $\mathcal{D_f((0, 1))}$ and $\mathcal{D_f((1, 0))}$.

b![b](assets\image-20250318000822800.png)



### Subproblem 1.3
Given $ p = \left( \frac{\pi}{4}, \frac{\pi}{6} \right) $ and $ v = (1, 0) $. For simplicity, let $ g_v(t) = f(\gamma(t)) $ denote the curve which passes through $ p $ at $ t = 0 $.
### Solution
#### (a) What is the arc length $ s(t) $ as the point moves from $ g_v(0) $ to $ g_v(t) $?

$$
s(t) = \int_0^t \|g_v'(t)\| dt = \int_0^t \|f'(\gamma(t))\| \nabla \gamma(t) dt = \int_0^t \|f'(\gamma(t))\| \mathbf{v} dt\\
= \int_0^t \|[-\sin u \sin v, \cos u \sin v, 0]^T\| dt
= \int_0^t |\sin v\| dt
= \frac{1}{2} t\\
$$
#### (b) Give the arc-length parametrization $ h_v(s) $ of the curve.
$$
h_v(s) = g_v(t(s)) = g_v(2s) = f(\gamma(2s)) = \\
[\frac{1}{2}\cos (\frac{\pi}{4} + 2s), \frac{1}{2}\sin (\frac{\pi}{4} + 2s), \frac{\sqrt{3}}{4}]^T
$$
#### (c) What is the normal vector of the curve at a point $ h_v(s) $? Hint: Use $ h_v(s) $ to derive the normal.
By the relation of normal of tangent vector of a curve $N(s) = \frac{T'(s)}{\|T'(s)\|}$, we can derive that:
$$
T(s) = h_v'(s) = [-\sin (\frac{\pi}{4} + 2s) \cos (\frac{\pi}{4} + 2s), 0]^T\\
T'(s) = [-2 \cos (\frac{\pi}{4} + 2s), -2\sin (\frac{\pi}{4} + 2s), 0]^T\\
N(s) = \frac{T'(s)}{\|T'(s)\|} = [- \cos (\frac{\pi}{4}+ 2s), - \sin( \frac{\pi}{4}+ 2s), 0 ]^T
$$

### Subproblem 1.4
Curvature: In 2(d), you have computed the normal at **p**. Denote this normal as $N_p$.

#### (a) Compute the differential of the normal $DN_p$, and express it as a matrix. 
From 
$$
N_p =
\begin{bmatrix}
\frac{1}{2}\cos u \sin^2 v \\
\frac{1}{2}\sin u \sin^2 v \\
\sin v \cos v
\end{bmatrix}
$$
By normalization:
$$
\hat N_p =
\frac{N_p}{\|N_p\|} =
\frac{-1}{\sqrt{\sin^2 v + 4\cos^2 v}} 
\begin{bmatrix}
\cos u \sin v\\
\sin u \sin v\\
2 \cos v
\end{bmatrix} \\
DN_p =  - [\frac{\partial \hat N_p}{\partial u}, \frac{\partial \hat N_p}{\partial v}]
=
- \begin{bmatrix}
\frac{\sin(u)\sin(v)}{\sqrt{\sin^2(v) + 4\cos^2(v)}} & -\frac{4\cos(u)\cos(v)}{(\sin^2(v) + 4\cos^2(v))^{3/2}},
 \\
-\frac{\cos(u)\sin(v)}{\sqrt{\sin^2(v) + 4\cos^2(v)}} & -\frac{4\sin(u)\cos(v)}{(\sin^2(v) + 4\cos^2(v))^{3/2}},
\\
0 & \frac{2\sin(v)}{(\sin^2(v) + 4\cos^2(v))^{3/2}}
\end{bmatrix} \in \mathbb{R}^{3 \times 2}
$$


#### (b) Find the eigenvectors of the shape operator at **p**.

From 
$$
Df_{\mathbf{p}} 
= 
\begin{bmatrix}
\frac{\partial f}{\partial u} & \frac{\partial f}{\partial v}
\end{bmatrix}
= \begin{bmatrix}
-a\sin u \sin v & a\cos u \cos v \\
b\cos u \sin v & b\sin u \cos v \\
0 & -c\sin v
\end{bmatrix}
= 
\begin{bmatrix}
-\sin u \sin v & \cos u \cos v \\
\cos u \sin v & \sin u \cos v \\
0 & -\frac{1}{2}\sin v
\end{bmatrix}
$$
And from the definition of the shape operator: 
$$
DN_{\mathbf{p}} = Df_{\mathbf{p}} S
$$
As the matrix $ Df_{\mathbf{p}} $ is not a square matrix, we use the pseudo-inverse of $Df_{\mathbf{p}}$:
$S = (Df_{\mathbf{p}}^T Df_{\mathbf{p}})^{-1} Df_{\mathbf{p}}^T DN_{\mathbf{p}}$
$$
Df_{\mathbf{p}}^T =
\begin{bmatrix}
-\sin u \sin v & \cos u \sin v & 0 \\
\cos u \cos v & \sin u \cos v & -\frac{1}{2}\sin v
\end{bmatrix}\\
Df_{\mathbf{p}}^T Df_{\mathbf{p}} =
\begin{bmatrix}
\sin^2 v & 0 \\
0 & \cos^2 v + \frac{1}{4}\sin^2 v
\end{bmatrix}\\
(Df_{\mathbf{p}}^T Df_{\mathbf{p}})^{-1} =
\begin{bmatrix}
\frac{1}{\sin^2 v} & 0 \\
0 & \frac{1}{\cos^2 v + \frac{1}{4}\sin^2 v}
\end{bmatrix}\\
S = (Df_{\mathbf{p}}^T Df_{\mathbf{p}})^{-1} Df_{\mathbf{p}}^T DN_{\mathbf{p}}
$$
$$
S = (Df_{\mathbf{p}}^T Df_{\mathbf{p}})^{-1} Df_{\mathbf{p}}^T DN_{\mathbf{p}} = 
\begin{bmatrix}
\frac{1}{\sqrt{\sin^2 v + 4\cos^2 v}} & 0 \\
0 & \frac{4}{(\sin^2 v + 4\cos^2 v)^{3/2}}
\end{bmatrix}
$$
We can derive that $S$ is diagonal, and its eigenvectors are $[0, 1]^T, [1, 0]$, which means the principal curvature directions are $[0, 1]^T, [1, 0]$.

#### (c) [Programming Assignment] Draw the two principal curvature directions in the tangent plane of the ellipsoid **p**
Actually I have done this in the previous question, and the result is shown in the figure below:
b![b](assets\image-20250318000822800.png) but you can also run the following code to get the result:

In [2]:
import numpy as np
import open3d as o3d
from src.draw import Ellipsoid, DifferentialMap

def draw_principal_curvature_directions():
    ellipsoid = Ellipsoid(a=1, b=1, c=0.5)
    mesh = ellipsoid.generate_surface()
    mesh.paint_uniform_color([0.8, 0.8, 0.8])
    wireframe = ellipsoid.get_wireframe()
    wireframe.paint_uniform_color([0.5, 0.5, 0.5])
    u, v = np.pi/4, np.pi/6
    x = ellipsoid.a * np.cos(u) * np.sin(v)
    y = ellipsoid.b * np.sin(u) * np.sin(v)
    z = ellipsoid.c * np.cos(v)
    point = np.array([x, y, z])
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector([point])
    point_cloud.paint_uniform_color([1, 0, 0]) 
    diff_map = DifferentialMap(ellipsoid, u, v)
    principal_directions = [
        ([1, 0], [1, 0, 0]),  
        ([0, 1], [0, 0, 1])   
    ]
    arrows = diff_map.create_differential_vectors(scale=0.3, directions=principal_directions)
    coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Principal Curvature Directions")
    vis.add_geometry(mesh)
    vis.add_geometry(wireframe)
    vis.add_geometry(point_cloud)
    vis.add_geometry(coordinate_frame)
    for arrow in arrows:
        vis.add_geometry(arrow)
    opt = vis.get_render_option()
    opt.background_color = np.array([1, 1, 1])  
    opt.point_size = 10.0  
    vis.run()
    vis.destroy_window()

if __name__ == "__main__":
    draw_principal_curvature_directions()

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


![](assets\2025-03-21-172541.png)

#### (d) Compute the Gaussian curvature of the surface **f** at **p** = ($\frac{\pi}{4}, \frac{\pi}{6}$), and demonstrate that ellipsoid **f** doesn’t show isometric invariance with any spherical surface.

From
$$
S = 
\begin{bmatrix}
\frac{1}{\sqrt{\sin^2 v + 4\cos^2 v}} & 0 \\
0 & \frac{4}{(\sin^2 v + 4\cos^2 v)^{3/2}}
\end{bmatrix}
$$
we get
$$
\lambda_1 = \frac{1}{\sqrt{\sin^2 v + 4\cos^2 v}}\\
\lambda_2 = \frac{4}{(\sin^2 v + 4\cos^2 v)^{3/2}}\\
K = \lambda_1 * \lambda_2 = \frac{4}{(\sin^2 v + 4\cos^2 v)^2}
$$
At $p = (\frac{\pi}{4}, \frac{\pi}{6})$
$$
K = \frac{4}{(\sin^2 \frac{\pi}{4} + 4\cos^2 \frac{\pi}{4})^2} = \frac{64}{169}
$$
From Gauss's Theorema Egregium, we know that the Gaussian curvature of an embedded smooth surface in $ \mathbb{R}^3 $ is invariant under local isometries. For the ellipsoid $ \mathbf{f} $, the Gaussian curvature is computed as $ K = \frac{4}{(\sin^2 v + 4\cos^2 v)^2} $, which explicitly depends on the parameter $ v $. At $ \mathbf{p} = (\frac{\pi}{4}, \frac{\pi}{6}) $, substituting $ v = \frac{\pi}{6} $, we find $ K = \frac{64}{169} $. However, this curvature is non-constant across the surface: for instance, at $ v = 0 $, $ K = \frac{1}{4} $, and at $ v = \frac{\pi}{2} $, $ K = 4 $. In contrast, any spherical surface has constant Gaussian curvature $ \frac{1}{R^2} $ determined solely by its radius $ R $. Since local isometries preserve Gaussian curvature invariantly, the ellipsoid’s variable curvature directly contradicts the sphere’s constant curvature, proving that no isometric equivalence can exist between them.
